In [38]:
import dash
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output
import pandas as pd
import plotly.graph_objects as go

# Load the dataset from Excel file
df = pd.read_excel(r'C:\Users\Abraha\Desktop\cleaned_month_shortened.xlsx')

# Remove rows where 'Short_Location', 'Month', or 'Short_Category' are null
df = df.dropna(subset=['Short_Location', 'Month', 'Short_Category'])

# Initialize the Dash app
app = dash.Dash(__name__)

# Create a mapping for labels without the "Short" prefix
label_mapping = {
    'Short_Assignment Group': 'Assignment Group',
    'Short_Category': 'Category',
    'Short_Service': 'Service',
    'Short_Issue': 'Issue',
    'Short_Location': 'Location'
}

# Define the layout with dropdowns for each filter
app.layout = html.Div([
    # Styled Header with fade-in effect
    html.H1(
        'Incidents Analysis Dashboard',  # New title
        className='fade-in',  # Applying fade-in effect to header
        style={
            'textAlign': 'center',  # Center-align the title
            'fontFamily': 'Arial, sans-serif',  # Use a clean sans-serif font
            'fontSize': '36px',  # Increase the font size for better visibility
            'fontWeight': 'bold',  # Make the title bold for emphasis
            'color': '#2E3B4E',  # Use a professional and modern color
            'marginBottom': '30px'  # Add some space below the title
        }
    ),

    # Div to contain dropdown menus side by side with slide-up effect
    html.Div([
        html.Div([
            html.Label(label_mapping['Short_Assignment Group']),
            dcc.Dropdown(
                id='assignment_group_dropdown',
                options=[{'label': i, 'value': i} for i in ['All'] + df['Short_Assignment Group'].unique().tolist()],
                multi=True,
                placeholder="Select Assignment Group",
                value=[]
            ),
        ], style={'width': '15%', 'padding': '5px', 'display': 'inline-block', 'animation': 'slideUp 0.5s ease-out', 'borderRight': '2px solid #ddd'}),

        html.Div([
            html.Label(label_mapping['Short_Category']),
            dcc.Dropdown(
                id='category_dropdown',
                options=[{'label': i, 'value': i} for i in ['All'] + df['Short_Category'].unique().tolist()],
                multi=True,
                placeholder="Select Category",
                value=[]
            ),
        ], style={'width': '15%', 'padding': '5px', 'display': 'inline-block', 'animation': 'slideUp 0.5s ease-out', 'borderRight': '2px solid #ddd'}),

        html.Div([
            html.Label(label_mapping['Short_Service']),
            dcc.Dropdown(
                id='service_dropdown',
                options=[{'label': i, 'value': i} for i in ['All'] + df['Short_Service'].unique().tolist()],
                multi=True,
                placeholder="Select Service",
                value=[]
            ),
        ], style={'width': '15%', 'padding': '5px', 'display': 'inline-block', 'animation': 'slideUp 0.5s ease-out', 'borderRight': '2px solid #ddd'}),

        html.Div([
            html.Label(label_mapping['Short_Issue']),
            dcc.Dropdown(
                id='issue_dropdown',
                options=[{'label': i, 'value': i} for i in ['All'] + df['Short_Issue'].unique().tolist()],
                multi=True,
                placeholder="Select Issue",
                value=[]
            ),
        ], style={'width': '15%', 'padding': '5px', 'display': 'inline-block', 'animation': 'slideUp 0.5s ease-out', 'borderRight': '2px solid #ddd'}),

        html.Div([
            html.Label(label_mapping['Short_Location']),
            dcc.Dropdown(
                id='location_dropdown',
                options=[{'label': i, 'value': i} for i in ['All'] + df['Short_Location'].unique().tolist()],
                multi=True,
                placeholder="Select Location",
                value=[]
            ),
        ], style={'width': '15%', 'padding': '5px', 'display': 'inline-block', 'animation': 'slideUp 0.5s ease-out', 'borderRight': '2px solid #ddd'}),

        html.Div([
            html.Label('Month'),
            dcc.Dropdown(
                id='month_dropdown',
                options=[{'label': i, 'value': i} for i in df['Month'].unique().tolist()],
                multi=True,
                placeholder="Select Month",
                value=[]
            ),
        ], style={'width': '15%', 'padding': '5px', 'display': 'inline-block', 'animation': 'slideUp 0.5s ease-out'}),
    ], style={'display': 'flex', 'flexWrap': 'wrap', 'justifyContent': 'space-between', 'marginBottom': '30px'}),

    # Heatmap placeholders with thin borders between sections
    html.Div([
        html.Div([
            dcc.Loading(children=[dcc.Graph(id='heatmap', className='fade-in')]),
        ], style={'borderBottom': '2px solid #ddd', 'paddingBottom': '30px'}),

        html.Div([
            dcc.Loading(children=[dcc.Graph(id='heatmap_service_vs_month', className='fade-in')]),
        ], style={'borderBottom': '2px solid #ddd', 'paddingBottom': '30px'}),

        html.Div([
            dcc.Loading(children=[dcc.Graph(id='heatmap_service_vs_location', className='fade-in')]),
        ], style={'borderBottom': '2px solid #ddd', 'paddingBottom': '30px'}),

        html.Div([
            dcc.Loading(children=[dcc.Graph(id='heatmap_location_vs_month', className='fade-in')]),
        ])
    ], style={'marginLeft': '10%', 'marginRight': '10%'})  # Add margins to heatmap section

])

# Define callback to update heatmaps based on dropdown selections
@app.callback(
    [Output('heatmap', 'figure'),
     Output('heatmap_service_vs_month', 'figure'),
     Output('heatmap_service_vs_location', 'figure'),
     Output('heatmap_location_vs_month', 'figure')],
    [
        Input('assignment_group_dropdown', 'value'),
        Input('category_dropdown', 'value'),
        Input('service_dropdown', 'value'),
        Input('issue_dropdown', 'value'),
        Input('location_dropdown', 'value'),
        Input('month_dropdown', 'value')
    ]
)
def update_heatmaps(assignment_group, category, service, issue, location, month):
    # Filter the DataFrame based on selected dropdown values
    filtered_df = df

    # Handle 'All' selections
    if assignment_group and 'All' not in assignment_group:
        filtered_df = filtered_df[filtered_df['Short_Assignment Group'].isin(assignment_group)]
    if category and 'All' not in category:
        filtered_df = filtered_df[filtered_df['Short_Category'].isin(category)]
    if service and 'All' not in service:
        filtered_df = filtered_df[filtered_df['Short_Service'].isin(service)]
    if issue and 'All' not in issue:
        filtered_df = filtered_df[filtered_df['Short_Issue'].isin(issue)]
    if location and 'All' not in location:
        filtered_df = filtered_df[filtered_df['Short_Location'].isin(location)]
    if month:
        filtered_df = filtered_df[filtered_df['Month'].isin(month)]

    # If no rows match the filter, return empty heatmaps
    if filtered_df.empty:
        return (
            {'data': [], 'layout': {'title': 'No data available for selected filters'}},  # Service vs Issue
            {'data': [], 'layout': {'title': 'No data available for selected filters'}},  # Service vs Month
            {'data': [], 'layout': {'title': 'No data available for selected filters'}},  # Service vs Location
            {'data': [], 'layout': {'title': 'No data available for selected filters'}}   # Location vs Month
        )

    # Create the Service vs Issue heatmap data
    heatmap_data_issue = filtered_df.groupby(['Short_Service', 'Short_Issue']).size().reset_index(name='Count')
    heatmap_issue = go.Figure(data=go.Heatmap(
        x=heatmap_data_issue['Short_Service'],
        y=heatmap_data_issue['Short_Issue'],
        z=heatmap_data_issue['Count'],
        colorscale='YlGnBu',
        text=heatmap_data_issue['Count'],
        texttemplate='%{text}',  # Show the count value as text inside the heatmap
        hovertemplate='<b>Service:</b> %{x}<br><b>Issue:</b> %{y}<br><b>Count:</b> %{z}',  # Custom hover template
        colorbar={'title': 'Count'},
        showscale=True,  # Enable color scale for the heatmap
        hoverongaps=False,
        name=''  # This removes the trace name ('trace 0')
    ))

    # Create the Service vs Month heatmap data
    heatmap_data_month = filtered_df.groupby(['Short_Service', 'Month']).size().reset_index(name='Count')
    heatmap_month = go.Figure(data=go.Heatmap(
        x=heatmap_data_month['Short_Service'],
        y=heatmap_data_month['Month'],
        z=heatmap_data_month['Count'],
        colorscale='YlGnBu',
        text=heatmap_data_month['Count'],
        texttemplate='%{text}',  # Show the count value as text inside the heatmap
        hovertemplate='<b>Service:</b> %{x}<br><b>Month:</b> %{y}<br><b>Count:</b> %{z}',  # Custom hover template
        colorbar={'title': 'Count'},
        showscale=True,
        hoverongaps=False,
        name=''  # This removes the trace name ('trace 0')
    ))

    # Create the Service vs Location heatmap data
    heatmap_data_location = filtered_df.groupby(['Short_Service', 'Short_Location']).size().reset_index(name='Count')
    heatmap_location = go.Figure(data=go.Heatmap(
        x=heatmap_data_location['Short_Service'],
        y=heatmap_data_location['Short_Location'],
        z=heatmap_data_location['Count'],
        colorscale='YlGnBu',
        text=heatmap_data_location['Count'],
        texttemplate='%{text}',  # Show the count value as text inside the heatmap
        hovertemplate='<b>Service:</b> %{x}<br><b>Location:</b> %{y}<br><b>Count:</b> %{z}',  # Custom hover template
        colorbar={'title': 'Count'},
        showscale=True,
        hoverongaps=False,
        name=''  # This removes the trace name ('trace 0')
    ))

    # Create the Location vs Month heatmap data
    heatmap_data_location_month = filtered_df.groupby(['Short_Location', 'Month']).size().reset_index(name='Count')
    heatmap_location_month = go.Figure(data=go.Heatmap(
        x=heatmap_data_location_month['Short_Location'],
        y=heatmap_data_location_month['Month'],
        z=heatmap_data_location_month['Count'],
        colorscale='YlGnBu',
        text=heatmap_data_location_month['Count'],
        texttemplate='%{text}',  # Show the count value as text inside the heatmap
        hovertemplate='<b>Location:</b> %{x}<br><b>Month:</b> %{y}<br><b>Count:</b> %{z}',  # Custom hover template
        colorbar={'title': 'Count'},
        showscale=True,
        hoverongaps=False,
        name=''  # This removes the trace name ('trace 0')
    ))

    return heatmap_issue, heatmap_month, heatmap_location, heatmap_location_month

if __name__ == '__main__':
    app.run_server(debug=True)